# Modelos *Ensemble*

Una de las tendencias más recientes en el ámbito del modelado de inteligencia artificial puede resumirse bajo la expresión **“el conocimiento del conjunto o de la multitud”**. Esta formulación, relativamente familiar, hace referencia al empleo de una multiplicidad de modelos denominados *débiles* en el contexto de un meta-clasificador. El propósito de este enfoque es generar un modelo *fuerte* a partir del conocimiento extraído por dichos modelos *débiles*.  

Por ejemplo, aunque se detallará con mayor profundidad más adelante, en un **Random Forest** se desarrollan múltiples Árboles de Decisión de menor complejidad. La combinación de estos en el bosque aleatorio permite alcanzar un rendimiento superior al de cualquiera de los modelos individuales considerados por separado. Los modelos que se construyen de esta manera, ya sea en forma de meta-clasificadores o meta-regresores, se denominan de forma genérica **modelos *Ensemble***.  

Conviene destacar que estos modelos no se limitan exclusivamente al uso de árboles de decisión, sino que pueden estar conformados por cualquier tipo de modelo de aprendizaje automático previamente estudiado. Incluso es posible construir **modelos mixtos**, en los que no todos los submodelos se hayan obtenido mediante el mismo procedimiento, sino a través de la combinación de diversas técnicas, tales como *k*-NN, *SVM*, entre otras.  

De este modo, el **primer criterio para clasificar los modelos *Ensemble*** consiste en determinar si se trata de modelos **homogéneos** o **heterogéneos**. No obstante, este no es el único criterio de clasificación. En esta unidad se abordarán diferentes estrategias para la generación de modelos y para su combinación posterior. Asimismo, se analizarán en detalle dos de las técnicas más representativas dentro de los modelos *Ensemble*: **Random Forest** , **_XGBoost_** y otras de las últimas tendencias como **LightGBM** y **CatBoost**.  


En primer lugar, asegúrate de que los paquetes necesarios estén instalados. Por tanto, esta celda debe ejecutarse únicamente la **primera vez**.


In [ ]:
using Pkg
# Asegúrate de que los paquetes requeridos estén disponibles  
#(descomenta las líneas correspondientes para instalarlos, si es necesario).
Pkg.add([
    "MLJ", 
    "MLJBase", 
    "MLJModels", 
    "MLJEnsembles", 
    "MLJLinearModels", 
    "DecisionTree", 
    "MLJDecisionTreeInterface", 
    "NaiveBayes", 
    "EvoTrees", 
    "CategoricalArrays", 
    "Random",
    "LIBSVM",           
    "Plots",            
    "MLJModelInterface", 
    "CSV",              
    "DataFrames",       
    "UrlDownload",      
    "XGBoost"    
])

## Preparación de los datos

A diferencia de los primeros tutoriales, en los que se empleó el problema de las flores *iris* como referencia, en este tutorial utilizaremos un caso distinto. El problema también se encuentra disponible en el repositorio UCI. Aunque se trata igualmente de un conjunto de datos de pequeño tamaño, el número de variables aumenta de forma significativa, lo que nos permitirá realizar un análisis más amplio.  

En concreto, se trata de un problema clásico de aprendizaje automático conocido informalmente como **¿Roca o Mina?** (*Rock or Mine?*). Este conjunto de datos contiene 111 patrones correspondientes a rocas y 97 a minas submarinas (simuladas como cilindros metálicos). Cada patrón está formado por **60 medidas numéricas** que representan una sección de las secuencias de sonar. Estos valores se encuentran ya en el rango de 0.0 a 1.0, aunque resulta recomendable normalizarlos por seguridad. Las medidas reflejan el valor energético de diferentes rangos de longitud de onda durante un determinado periodo temporal.  

En este proceso vamos a emplear un par de paquetes nuevos, concretamente [DataFrames.jl](https://juliaai.github.io/DataScienceTutorials.jl/data/dataframe/) y [UrlDownload.jl](https://github.com/Arkoniak/UrlDownload.jl).  
Por tanto, lo primero que debemos hacer es asegurarnos de que los paquetes estén correctamente instalados.


In [ ]:
using Pkg;
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("UrlDownload")

A continuación, los datos se descargarán en caso de que no estén disponibles localmente.  
Para ello, puede emplearse el siguiente fragmento de código:

In [ ]:
using UrlDownload
using DataFrames
using CSV
using CategoricalArrays

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
data = urldownload(url, true, format=:CSV, header=false) |> DataFrame
describe(data)

Como puede observarse en la línea anterior, hemos descargado los datos y los hemos canalizado mediante el operador `|>` hacia la función `DataFrame`.  
Esto permite crear una estructura similar a una tabla de base de datos, lo cual resulta especialmente conveniente para comprobar la existencia de valores ausentes o analizar los rangos de las distintas variables.  

De hecho, esta biblioteca facilita notablemente la gestión de valores perdidos, al proporcionar funciones que permiten tanto completar como eliminar las muestras con medidas no válidas.  
No obstante, el conjunto de datos es demasiado extenso para visualizar todas las variables en el informe de salida. Si realizamos algunas consultas específicas, podemos comprobar que **no existen valores ausentes**. Además, ninguna variable supera el valor de 1.0, aunque algunas de ellas no están completamente normalizadas.  

Una estructura análoga puede encontrarse en otros lenguajes de programación, como **R** o **Python**.  

Como ejemplo de este proceso, añadiremos una columna adicional con el fin de **convertir en categórica la última columna (la número 60)**, que contiene una **M** para cada muestra correspondiente a una mina y una **R** para cada muestra correspondiente a una roca.


In [ ]:
insertcols!(data, :Mine => data[:, 61].=="M")

Una vez que los datos se han cargado en el `DataFrame` para su verificación y se ha aplicado cualquier posible proceso de preparación, es necesario transformarlos para su utilización en los modelos.  

Al igual que en los tutoriales anteriores, los datos deben convertirse a una **forma matricial**, tal como se muestra a continuación:


In [ ]:
input_data = Matrix(data[!, 1:60]);
output_data = data[!, :Mine];

@assert input_data isa Matrix
@assert output_data isa BitVector

Cabe destacar que, en un `DataFrame`, cuando se consulta un conjunto de filas, como en el caso de la variable `X`, el resultado obtenido es también un objeto de tipo `DataFrame`.  

Por tanto, para poder aplicar las operaciones posteriores, es necesario utilizar la función `Matrix`, la cual convierte dicho resultado en una **matriz** sobre la que pueden ejecutarse las operaciones habituales.


### Pregunta

> ❓Ahora, los datos se han cargado y convertido a los tipos habituales. A continuación, deberías poder abordar la siguiente sección: realizar una partición del conjunto de datos en dos subconjuntos, **entrenamiento** y **prueba**, y aplicar la normalización correspondiente. Incluye el código en la sección siguiente para ejecutar ambas operaciones. *Consejo: debido a la preparación para los modelos de MLJ, lee las notas al final del documento.*


In [ ]:
#train_input, train_output, test_input, test_output = #TODO


## Línea base

Como se ha mencionado, los *ensembles* son conjuntos de clasificadores “más débiles” que, al combinarse, permiten superar sus limitaciones individuales. Por ello, antes de abordar los *ensembles*, resulta necesario disponer de algunos **modelos de referencia** que posteriormente se integrarán en un **meta-clasificador**.

En el ejemplo siguiente, se entrenan varios modelos sencillos implementados con la biblioteca `MLJ`:  
- una **SVM** con **núcleo RBF**  
- una **Regresión Lineal**  
- un **Naïve Bayes**  
- un **Árbol de Decisión**.

In [ ]:

using MLJ
using MLJBase: accuracy

# Cargar los modelos (tenga presente que si algunos no están instalados, MLJ le pedirá que los instale)
SVC = @load ProbabilisticSVC pkg=LIBSVM
LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree
GaussianNBClassifier = @load GaussianNBClassifier pkg=NaiveBayes

#Definir los modelos base
models = Dict(
    "SVM" => SVC(),
    "LR"  => LogisticClassifier(),
    "DT"  => DecisionTreeClassifier(max_depth=4),
    "NB"  => GaussianNBClassifier(),
)

base_models=  [ model for (name, model) in models]

machines = Dict()

In [ ]:
# Ejecutar el entrenamiento de cada modelo y calculas los valores de test (accuracy)
for (name, model) in models
    machines[name] = machine(model, train_input, train_output) |> fit!
    acc = MLJ.accuracy(predict_mode(machines[name], test_input), test_output)
    println("$name: $(acc*100) %")
end

## Combinación de modelos débiles en un *ensemble*

A la hora de combinar los modelos, existen diversas estrategias que dependen de la tarea que se esté abordando, es decir, de si se trata de un problema de **clasificación** o de **regresión**. En este caso concreto nos centraremos en la clasificación; no obstante, para la regresión el procedimiento sería similar, aunque habría que tener en cuenta la naturaleza continua de los valores al combinar las salidas.

En lo que respecta a la **combinación de clasificadores**, existen principalmente dos enfoques para integrar las salidas de varios modelos:  
- **Votación mayoritaria** (*Majority Voting*), y  
- **Votación mayoritaria ponderada**, también conocida como **Votación Suave** (*Soft Voting*).


Como primer paso y dado que MLJ no dispone de una clase para este tipo de clasificaciones de manera nativa, vamos a definir una que soporte modelos heterogéneos

In [ ]:
using MLJ
using MLJBase
using MLJModelInterface

# ===================================================
# DEFINICION del VOTINGCLASSIFIER compatible con MLJ
# ===================================================

"""
    VotingClassifier <: Probabilistic

Un clasificador *ensemble* que combina las predicciones de múltiples modelos base utilizando diferentes estrategias de votación.

# Campos
- `models::Vector{Probabilistic}`: Vector de modelos probabilísticos base que se combinarán.  
- `voting::Symbol`: Estrategia de votación, que puede ser `:hard` (votación mayoritaria) o `:soft` (promedio de probabilidades).  
- `weights::Union{Nothing, Vector{Float64}}`: Pesos opcionales para cada modelo. Si se establece como `nothing`, todos los modelos tendrán el mismo peso. Los pesos se normalizan automáticamente para que su suma sea 1.0.

# Ejemplos
```julia
# Pesos iguales (por defecto)
voting_clf = VotingClassifier(
    models=[LogisticClassifier(), DecisionTreeClassifier()],
    voting=:soft
)

# Pesos personalizados (se normalizan automáticamente)
voting_clf = VotingClassifier(
    models=[LogisticClassifier(), DecisionTreeClassifier(), RandomForestClassifier()],
    voting=:hard,
    weights=[5, 3, 2]  # Se normalizarán a [0.5, 0.3, 0.2]
)
mutable struct VotingClassifier <: Probabilistic   # Models must be probabilistic, inherited from MLJBase
    models::Vector{Probabilistic}
    voting::Symbol  # :hard or :soft
    weights::Union{Nothing, Vector{Float64}}
end
```
"""

"""
    VotingClassifier(; models=Probabilistic[], voting=:hard, weights=nothing)
Constructor del `VotingClassifier`.

# Argumentos
- `models::Vector{Probabilistic}=Probabilistic[]`: Modelos base que se combinarán.  
- `voting::Symbol=:hard`: Estrategia de votación (`:hard` o `:soft`).  
- `weights::Union{Nothing, Vector{<:Real}}=nothing`: Pesos asignados a cada modelo. Se normalizan automáticamente para que su suma sea 1.0.

# Excepciones
- `AssertionError`: Si el parámetro `voting` no es `:hard` ni `:soft`.  
- `AssertionError`: Si la longitud del vector de pesos no coincide con el número de modelos.  
- `AssertionError`: Si todos los pesos son cero o negativos.
"""

function VotingClassifier(; models=Probabilistic[], voting=:hard, weights=nothing)
    @assert voting in [:hard, :soft] "The only possible labels are :hard or :soft"
    
    normalized_weights = nothing
    if weights !== nothing
        @assert length(weights) == length(models) "Number of weights must match number of models"
        @assert all(w >= 0 for w in weights) "All weights must be non-negative"
        
        # Normalizar los pesos para sumar 1.0
        normalized_weights = Float64.(weights) ./ sum(weights)
    end
    
    return VotingClassifier(models, voting, normalized_weights)
end

"""
    MLJModelInterface.fit(model::VotingClassifier, verbosity::Int, X, y)

Entrena el `VotingClassifier` ajustando cada modelo base con los datos proporcionados.

# Argumentos
- `model::VotingClassifier`: Instancia del clasificador de votación.  
- `verbosity::Int`: Nivel de verbosidad para el registro del proceso de entrenamiento.  
- `X`: Características de entrenamiento (en formato de tabla).  
- `y`: Variable objetivo de entrenamiento (vector categórico).

# Retorna
- `fitresults`: Vector de máquinas entrenadas (una por cada modelo base).  
- `cache`: `nothing` (no se implementa almacenamiento en caché).  
- `report`: Tupla con nombre que contiene información del entrenamiento (número de modelos, estrategia de votación y pesos normalizados).
"""

function MLJModelInterface.fit(model::VotingClassifier, verbosity::Int, X, y)
    # Entrenar cada modelo base
    fitresults = []
    for base_model in model.models
        model_copy = deepcopy(base_model)
        mach = machine(model_copy, X, y)
        fit!(mach, verbosity=0)
        push!(fitresults, mach)
    end
    
    # Guardar información necesaria para el reporte sobre el entrenamiento
    cache = nothing
    report = (n_models=length(model.models), voting=model.voting, weights=model.weights)
    
    return fitresults, cache, report
end

"""
    MLJModelInterface.predict_mode(model::VotingClassifier, fitresult, Xnew)

Predice las etiquetas de clase utilizando **votación dura** (*hard voting*), es decir, votación mayoritaria con pesos opcionales.

# Argumentos
- `model::VotingClassifier`: Instancia del clasificador de votación.  
- `fitresult`: Vector de máquinas entrenadas obtenido en la fase de ajuste.  
- `Xnew`: Nuevos datos sobre los que se realizará la predicción.

# Retorna
- Vector categórico con las etiquetas de clase predichas, calculadas mediante votación mayoritaria (ponderada o no).

# Detalles
Cada modelo base emite un voto por una clase.  
Si se han definido pesos, cada voto se multiplica por el peso correspondiente.  
La clase con el mayor número de votos (ponderados) es seleccionada como predicción final.
"""

function MLJModelInterface.predict_mode(model::VotingClassifier, fitresult, Xnew)
    machines = fitresult
    
    # Calcular las predicciones de cada modelo base
    predictions = [predict_mode(mach, Xnew) for mach in machines]
    
    # Obtener todas las clases posibles
    all_classes = unique(vcat([unique(p) for p in predictions]...))
    n_samples = length(predictions[1])
    n_models = length(machines)
    
    # Determinar los pesos (iguales si no se especifican)
    weights = model.weights === nothing ? fill(1.0/n_models, n_models) : model.weights
    
    # Votación mayoritaria ponderada
    ensemble_pred = Vector{eltype(predictions[1])}(undef, n_samples)
    
    for i in 1:n_samples
        # Contar los votos ponderados para cada clase
        vote_counts = Dict{eltype(predictions[1]), Float64}()
        for class in all_classes
            vote_counts[class] = 0.0
        end
        
        for (j, pred) in enumerate(predictions)
            vote_counts[pred[i]] += weights[j]
        end
        
        # Seleccionar la clase con el mayor número de votos
        ensemble_pred[i] = argmax(vote_counts)
    end
    
    return categorical(ensemble_pred)
end

"""
    MLJModelInterface.predict(model::VotingClassifier, fitresult, Xnew)

Predice las probabilidades de clase utilizando la estrategia de votación especificada.

# Argumentos
- `model::VotingClassifier`: Instancia del clasificador de votación.  
- `fitresult`: Vector de máquinas entrenadas obtenido durante el ajuste.  
- `Xnew`: Nuevos datos sobre los que se realizarán las predicciones.

# Retorna
- Vector de distribuciones `UnivariateFinite` que representan las probabilidades de pertenencia a cada clase.

# Detalles
- Para la votación `:hard`: se devuelven predicciones deterministas encapsuladas en `UnivariateFinite` (con pesos opcionales).  
- Para la votación `:soft`: se calculan las probabilidades promediando las distribuciones generadas por todos los modelos base, aplicando los pesos correspondientes.
"""

function MLJModelInterface.predict(model::VotingClassifier, fitresult, Xnew)
    machines = fitresult
    
    result = if model.voting == :hard
        # Para hard voting, devvuelve las predicciones tal cual
        UnivariateFinite(predict_mode(model, fitresult, Xnew))
    else
        # Para soft voting, promedia las probabilidades con los pesos
        all_predictions = [predict(mach, Xnew) for mach in machines]
        
        # Recupera la información de las clases
        first_pred = all_predictions[1][1]
        class_levels = MLJBase.classes(first_pred)
        n_classes = length(class_levels)
        n_samples = length(all_predictions[1])
        n_models = length(machines)
        
        # Determina los pesos (iguales si no se especifican)
        weights = model.weights === nothing ? fill(1.0/n_models, n_models) : model.weights
        
        # Calcula la media ponderada de las probabilidades
        avg_probs = zeros(n_samples, n_classes)
        for (model_idx, preds) in enumerate(all_predictions)
            for i in 1:n_samples
                for (j, level) in enumerate(class_levels)
                    avg_probs[i, j] += weights[model_idx] * pdf(preds[i], level)
                end
            end
        end
        
        # Crea distribuciones `UnivariateFinite` con probabilidades promediadas de forma ponderada.
        [UnivariateFinite(class_levels, avg_probs[i, :]) for i in 1:n_samples]
    end
    
    return result
end

"""
Registro de metadatos del modelo para `VotingClassifier`.

Especifica los tipos de entrada/salida y las capacidades para su integración con MLJ.
"""
MLJModelInterface.metadata_model(VotingClassifier,
    input_scitype=Table(Continuous),
    target_scitype=AbstractVector{<:Finite},
    supports_weights=false,
    load_path="VotingClassifier"
)

## Votación mayoritaria

También conocida como *Hard Voting*, consiste, como su nombre sugiere, en seleccionar la opción más votada entre las predicciones emitidas por los distintos modelos. Cada modelo emite un voto o predicción determinista. La clase (o predicción) final es aquella que recibe **el mayor número de votos** o, en caso de regresión, **el valor medio** entre los resultados. Es equivalente a una “elección democrática” en la que cada modelo/experto dispone de un voto y gana la opción más votada. De este modo, el problema puede abordarse teniendo en cuenta diferentes resultados o puntos de vista.

### Ejemplo

Con 3 clasificadores que predicen un mismo patrón:

- **SVM** predice: **Mina**  
- **Regresión Logística** predice: **Roca**  
- **Naive Bayes** predice: **Roca**

**Resultado:** **Roca** (2 votos frente a 1) ✓

A continuación se muestra un ejemplo en código de cómo construir un modelo de este tipo.


In [ ]:
#Define el **meta-clasificador** a partir de los `base_models`.

models["Ensemble (Hard Voting)"] = VotingClassifier(estimators = base_models, voting=:hard)
machines["Ensemble (Hard Voting)"] = machine(models["Ensemble (Hard Voting)"], train_input, train_output) |> fit!

for (name, machine) in machines
    acc = MLJ.accuracy(predict_mode(machine, test_input), test_output)
    println("$name: $(acc*100) %")
end

El problema principal radica en que **confiamos por igual en todos los modelos** a la hora de decidir la clase de respuesta.

## *Soft Voting* 

Como se mencionó en la sección anterior, uno de los problemas de los modelos *ensemble* clásicos es que **todos los resultados se ponderan de igual manera** y que, en cada uno de los modelos “débiles”, solo se tiene en cuenta la clase más votada.  

Para resolver esta limitación, el *Soft Voting* propone utilizar las **probabilidades** que cada clasificador asigna a cada clase, en lugar de considerar únicamente la clase predicha. El resultado final se obtiene promediando dichas probabilidades y seleccionando la clase con la **mayor probabilidad promedio**.

### Ejemplo sin pesos (todos los modelos igualmente importantes)

| Clasificador | P(Mina) | P(Roca) |
|---------------|---------|---------|
| SVM           | 0.9     | 0.1     |
| Regresión Logística | 0.3 | 0.7 |
| Naive Bayes   | 0.2     | 0.8 |
| **Promedio**  | **0.47**| **0.53** |

**Cálculo:**
- P(Mina) = (0.9 + 0.3 + 0.2) / 3 = 0.47  
- P(Roca) = (0.1 + 0.7 + 0.8) / 3 = 0.53  

**Resultado:** **Roca** (mayor probabilidad promedio) ✓  

**Ventaja frente al _Hard Voting_:**  
Aunque el modelo SVM muestra una alta confianza en la clase *Mina* (0.9), los otros dos modelos manifiestan una confianza significativa en *Roca* (0.7 y 0.8). La votación suave permite **capturar y aprovechar esta información de confianza** en la combinación final.


In [ ]:
#Define el **meta-clasificador** a partir de los `base_models`.

models["Ensemble (Soft Voting - Equal)"] = VotingClassifier( models = base_models, voting = :soft, weights = nothing) # All models equally weighted
machines["Ensemble (Soft Voting - Equal)"] = machine(models["Ensemble (Soft Voting - Equal)"], train_input, train_output) |> fit!
for (name, machine) in machines
    acc = MLJ.accuracy(predict_mode(machine, test_input), test_output)
    println("$name: $(acc*100) %")
end

#### *Weighted Soft Voting* (Votación ponderada)

Aunque *Soft Voting* mejora el proceso al tener en cuenta el grado de confianza de cada modelo, el principal inconveniente sigue siendo la **igualdad de influencia entre los modelos**.  
Para solucionar este aspecto, una de las propuestas consiste en **introducir un sistema de ponderación en la decisión**.  

En muchos casos sabemos que algunos modelos presentan un mejor rendimiento que otros.  
Por ejemplo, si una **SVM** alcanza una precisión del **85 %**, mientras que los demás modelos rondan el **70 %**, es razonable asignar **mayor peso** a la SVM.  
Esto se consigue mediante el uso de **pesos** en la votación suave, donde los valores de probabilidad de cada modelo se **multiplican por su peso** antes de calcular el promedio.  

Matemáticamente:

$$
P(\text{clase}) = \frac{\sum_{i=1}^{n} w_i \cdot P_i(\text{clase})}{\sum_{i=1}^{n} w_i}
$$

Donde:

* $w_i$ = peso asignado al modelo $i$  
* $P_i(\text{clase})$ = probabilidad asignada por el modelo $i$ a la clase  
* $n$ = número total de modelos  

Siguiendo el mismo ejemplo anterior, supongamos que deseamos **incrementar la importancia de la SVM**.

### Ejemplo con pesos [2, 1, 1] (peso doble para la SVM)

| Clasificador | Peso | P(Mina) | P(Roca) | Contribución Mina | Contribución Roca |
|---------------|------|----------|----------|-------------------|-------------------|
| SVM           | 2    | 0.9      | 0.1      | 2 × 0.9 = 1.8     | 2 × 0.1 = 0.2     |
| Reg. Logística| 1    | 0.3      | 0.7      | 1 × 0.3 = 0.3     | 1 × 0.7 = 0.7     |
| Naive Bayes   | 1    | 0.2      | 0.8      | 1 × 0.2 = 0.2     | 1 × 0.8 = 0.8     |
| **Suma**      | 4    |          |          | **2.3**           | **1.7**           |
| **Promedio ponderado** |  |    |          | **0.575**         | **0.425**         |

**Cálculo:**
- P(Mina) = (1.8 + 0.3 + 0.2) / 4 = 2.3 / 4 = 0.575  
- P(Roca) = (0.2 + 0.7 + 0.8) / 4 = 1.7 / 4 = 0.425  

**Resultado:** **Mina** (mayor probabilidad ponderada) ✓


In [ ]:
models["Ensemble (Soft Voting - Weighted)"] = VotingClassifier(estimators = base_models, voting=:soft,weights=[1,2,2,1])
machines["Ensembles (Soft Voting - Weighted)"] = machine(models["Ensemble (Soft Voting - Weighted)"],train_input, train_output) |> fit!

for (name, machine) in machines
    acc = MLJ.accuracy(predict_mode(machine, test_input), test_output)
    println("$name: $(acc*100) %")
end

## Cuándo utilizar cada estrategia

### *Hard Voting*

- Cuando los modelos solo generan predicciones categóricas (sin probabilidades).  
- Cuando todos los modelos tienen una fiabilidad similar.  
- Es un método más **simple y rápido**.

### *Soft Voting*

- Cuando los modelos proporcionan **probabilidades** como salida.  
- Cuando todos los modelos presentan **rendimientos comparables**.  
- Permite **capturar el grado de confianza** en cada predicción.

### *Weighted Soft Voting*

- Cuando algunos modelos son claramente **más precisos o robustos** que otros.  
- Cuando se desea **asignar mayor importancia** a determinados modelos.  
- Los pesos pueden definirse en función de:
    - La **precisión de validación**.  
    - La **especialización o experiencia** del modelo en un dominio concreto.  
    - Métricas relevantes como el **F1-score** u otras medidas de rendimiento.  

Para la **selección de los pesos**, pueden emplearse varias estrategias, siendo las más relevantes:

1. **Asignación manual**, basada en conocimiento previo del rendimiento de los modelos.  
2. **Basada en la precisión de validación**, ajustando los pesos en proporción a los resultados obtenidos.  
3. **Optimización mediante búsqueda en _Grid search_**), explorando distintas combinaciones de pesos para maximizar el rendimiento global del *ensemble*.


### Pregunta
> ❓Hemos realizado todas las pruebas utilizando una **estrategia *hold-out***; sin embargo, como se indicó en una sesión anterior, se prefiere la aplicación de un enfoque de **validación cruzada (*cross-validation*)** para reducir la dependencia de la selección de las muestras.  

En este caso, pueden considerarse dos enfoques diferentes:  

1. Aplicar la validación cruzada a cada modelo individualmente, elegir el mejor y combinarlos en un único *ensemble*.  
2. Aplicar la validación cruzada al nivel del *ensemble* antes de entrenar los modelos.  

¿Cuál de ellos es el correcto y por qué?

`Answer here`

### *Stacking*

Este último enfoque para la combinación de modelos puede considerarse una __variante *Soft Voting*__.   
Como se mencionó en dicha sección, la votación suave permite fijar los pesos de cada uno de los modelos, y estos pueden ajustarse mediante una técnica de **gradiente descendente**.  

El *Stacking* suele identificarse como una técnica de clasificación superior a una regresión lineal (que es, en esencia, lo que realiza la votación suave), al emplear un **modelo más complejo**, como una **red neuronal artificial (ANN)**, para combinar los resultados de los modelos base.  

Así, de manera análoga a lo realizado anteriormente, las salidas de las diferentes técnicas pueden tomarse y utilizarse como **entradas de otro modelo de clasificación**, lo que permite ajustar los pesos y considerar **combinaciones no lineales** de las respuestas de cada modelo.  

A continuación se muestra un ejemplo de este proceso en código, utilizando la implementación de `MLJ`, que emplea un **SVC como modelo combinador**:


In [ ]:
#Crea una `NamedTuple` con los modelos base.# 
base_models_NamedTuple = (; (Symbol(name) => model for (name, model) in models)...)

# Construir el stacking:
# - resampling=CV(...) define cómo se generan las predicciones out-of-fold
# - measures=... solo para reporte interno; no afecta al entrenamiento final del stack
models["Ensemble (Stacking)"] = Stack(; 
    metalearner = SVC(),
    resampling = CV(nfolds=5, shuffle=true, rng=123),
    measures = log_loss,
    base_models_NamedTuple...  # Expande la `NamedTuple` de modelos base.
)

# Entrenar el stack en el train dataset
machines["Ensemble (Stacking)"] = machine(stack, train_input, train_output) |> fit!

In [ ]:
for (name, machine) in machines
    acc = MLJ.accuracy(predict_mode(machine, test_input), test_output)
    println("$name: $(acc*100) %")
end

## Creación de modelos

Uno de los elementos clave que aún no se ha abordado es la creación de los modelos que compondrán el meta-clasificador. Hasta ahora, el enfoque seguido no es muy adecuado, ya que el conjunto de datos de entrada para todos los modelos es el mismo. Esto provoca una evidente falta de diversidad en los modelos, dado que, sea cual sea el modelo que creemos, dispondrá de la misma información o “punto de vista” que los demás. Sin embargo, esta no es la práctica habitual. En su lugar, el conjunto de patrones de entrada suele **dividirse en subconjuntos más pequeños** con los que entrenar una o varias técnicas, con el doble objetivo de **reducir el coste computacional** por un lado y **aumentar la diversidad de los modelos** por otro. Conviene recordar en este punto que los modelos “débiles” no tienen por qué ser perfectos en todas las clases ni cubrir todas las posibilidades; basta con que sean **rápidos de entrenar** y ofrezcan una salida **razonablemente consistente**.

En cuanto a la forma de particionar los datos para la creación de los modelos, la mayoría de los enfoques suelen considerar dos estrategias principales, conocidas como **_Bagging_** y **_Boosting_**. A continuación se describen brevemente ambas.

### Bagging o agregación *bootstrap*
La técnica conocida como _Bagging_ o selección con reemplazo fue propuesta por **Breitman** en 1996. Se basa en el desarrollo de múltiples modelos que pueden entrenarse en **paralelo**. El elemento clave de estos modelos es que **cada modelo se entrena sobre un subconjunto del conjunto de entrenamiento**. Este subconjunto de datos se extrae **aleatoriamente con reemplazo**. Este último punto es especialmente importante porque, una vez que se ha seleccionado un ejemplo de entre las posibilidades, se **vuelve a colocar** entre las posibilidades para que pueda ser seleccionado nuevamente, ya sea en el subconjunto que se está construyendo o en los subconjuntos de otros modelos; es decir, se crean **conjuntos de ejemplos no disjuntos**.

![Bagging Example](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Ensemble_Bagging.svg/440px-Ensemble_Bagging.svg.png)

El resultado es la creación de “expertos” en datos **especializados** y dependientes de la partición. Si bien los datos **comunes** o más frecuentes quedan correctamente cubiertos por todos los modelos, también es cierto que los datos **menos frecuentes** tienden a no estar presentes en todas las particiones y pueden no quedar cubiertos en todos los casos. De este modo, se obtienen modelos **más especializados** en ciertos datos o con un **punto de vista distinto**, que se comportan como expertos en una región particular del espacio de búsqueda.

Aunque se tratará con mayor detalle más adelante, una técnica bien conocida que emplea este enfoque para la construcción de sus modelos “débiles” es **RandomForest**. En efecto, construye los árboles de decisión que componen el meta-clasificador de esta manera. **Cualquier clasificador** puede utilizarse como base de un _Bagging_ mediante la clase [EnsembleModel](https://juliaai.github.io/MLJ.jl/stable/models/EnsembleModel_MLJEnsembles/#EnsembleModel_MLJEnsembles).

Por ejemplo, en el siguiente código se han elegido **10 SVM para clasificación** como modelos débiles. Cada uno de estos modelos se ha entrenado **solo con el 50 %** de los patrones de entrenamiento, por lo que debería **incrementarse la varianza** entre ellos.

 

In [ ]:
# Añade un modelo de *Bagging* utilizando **SVC** como modelo base.
using MLJEnsembles: EnsembleModel, CPUThreads

models["Bagging (SVC)"] = EnsembleModel(
    model = SVC(),              # o ProbabilisticSVC()
    n = 10,                     # numbero de modelos base 
    bagging_fraction = 0.50,    # fraction de eejemplos por modelo base
    rng = 123,                  
    acceleration = CPUThreads() # Utiliza `Threads` para acelerar el entrenamiento, dada la independencia de los modelos base.
)

machines["Bagging (SVC)"] = machine(models["Bagging (SVC)"], train_input, train_output) |> fit!

for (name, machine) in machines
    acc = MLJ.accuracy(predict_mode(machine, test_input), test_output)
    println("$name: $(acc*100) %")
end

Como alternativa a la extracción de ejemplos completos, podría realizarse una **partición vertical** del conjunto de entrenamiento, extrayendo así **características** (*features*).  
Para implementar esta alternativa, en la función `EnsembleModel` debe definirse el parámetro **`bagging_fraction`**.  

Este enfoque se utiliza especialmente cuando el número de características es muy elevado, con el objetivo de crear **modelos más simples** que no utilicen toda la información disponible, ya que esta suele contener redundancia.  

Es importante destacar que este procedimiento de extracción de características para los modelos se realiza **sin reemplazo**, es decir, las características seleccionadas para un clasificador **no se vuelven a incluir** en la lista de posibilidades hasta que se construye el conjunto correspondiente al siguiente clasificador.


### *Boosting*

La otra gran familia de técnicas para el meta-modelado *ensemble* es la conocida como **_Boosting_**.En este caso, el enfoque es ligeramente diferente, ya que el objetivo es crear una **cadena de clasificadores**.La idea clave es que cada clasificador sucesivo se vuelve **más especializado en los patrones que los modelos anteriores clasificaron incorrectamente**.  

Así, al igual que en el caso anterior, se selecciona un subconjunto de patrones del conjunto original, pero este proceso se realiza **de manera secuencial y sin reemplazo**.Esto provoca que los nuevos aprendices se centren progresivamente en los casos más difíciles, generando gradualmente un modelo compuesto más robusto y preciso.  

Por tanto, como en el *Bagging*, la idea subyacente de este enfoque es que **no todos los modelos deben utilizar todos los patrones como base**, pero, a diferencia del *Bagging*, este proceso es **lineal**, debido a la **dependencia entre la construcción de los modelos**. Finalmente, las salidas de los modelos individuales se combinan mediante una **votación mayoritaria ponderada**, donde el peso de cada clasificador refleja su rendimiento durante el entrenamiento.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Ensemble_Boosting.svg/1920px-Ensemble_Boosting.svg.png" alt="Boosting examples" width="600"/>

#### AdaBoost

El algoritmo **AdaBoost** comienza asignando **pesos iguales** a todas las instancias del conjunto de entrenamiento. A continuación, se entrena un clasificador sencillo (denominado *stump*, que consiste en un árbol de decisión de un solo nivel) y se evalúa su rendimiento. Las instancias mal clasificadas reciben **mayor peso**, de modo que el siguiente clasificador se centra más en esos casos difíciles.  

Este proceso iterativo continúa, actualizando los pesos en cada iteración y creando un nuevo modelo débil que complementa a los anteriores. En **AdaBoost**, la ponderación tanto de las instancias como de los clasificadores se basa en una **función de pérdida exponencial**, que penaliza las clasificaciones erróneas de manera exponencial. La predicción final del *ensemble* se obtiene mediante una **votación mayoritaria ponderada** entre todos los clasificadores débiles.  

En **MLJ**, este comportamiento está implementado mediante el modelo `AdaBoostStumpClassifier`, proporcionado por el paquete `DecisionTree.jl`.

#### Gradient Boosting

El **Gradient Boosting** sigue un principio distinto: en lugar de reajustar los pesos de las instancias de forma explícita, utiliza un enfoque de **descenso por gradiente** para minimizar una función de pérdida. Cada nuevo árbol de la secuencia se entrena para predecir los **errores residuales** (o gradientes) del *ensemble* anterior, refinando gradualmente el modelo.  

En el caso de la clasificación, cada árbol de decisión modela la **verosimilitud logística** de los datos, y sus predicciones se combinan para estimar las probabilidades de clase. La decisión final se obtiene a partir de la **suma de estas probabilidades** a lo largo de todos los árboles.  

En **MLJ**, este procedimiento puede implementarse mediante el modelo `EvoTreeClassifier` (del paquete `EvoTrees.jl`), que es conceptualmente similar al `GradientBoostingClassifier` de *scikit-learn*, aunque está escrito íntegramente en Julia y admite **aceleración tanto por CPU como por GPU**.


In [ ]:
using MLJ
using MLJBase: accuracy

# Carga modelos (puro Julia)
AdaBoostStumpClassifier = @load AdaBoostStumpClassifier pkg=DecisionTree
EvoTreeClassifier = @load EvoTreeClassifier pkg=EvoTrees

# AdaBoost (similar a sklearn AdaBoostClassifier con stumps)
models["AdaBoost"] = AdaBoostStumpClassifier(n_iter = 30)
machines["AdaBoost"] = machine(models["AdaBoost"], train_input, train_output) |> fit!(


# Gradient Boosting (similar a sklearn GradientBoostingClassifier)
models["EvoTrees"] = EvoTreeClassifier(
    nrounds=30,
    eta=1.0,
    max_depth=2,
    loss=:logistic
)

machines["EvoTrees"] = machine(models["EvoTrees"], train_input, train_output) |> fit!()


for (name, machine) in machines
    acc = MLJ.accuracy(predict_mode(machine, test_input), test_output)
    println("$name: $(acc*100) %")
end


### Pregunta
> De forma análoga a la sección de validación cruzada, desarrolla una función para entrenar *ensembles*.  
La función, denominada `trainClassEmsemble`, seguirá también una **validación cruzada estratificada**.  
A modo de recordatorio rápido, los pasos que debe cubrir la función son:

1. **Crear un vector con *k* elementos**, que contendrá los resultados en test del proceso de validación cruzada con la métrica seleccionada.  

2. **Realizar un bucle de *k* iteraciones** (*k* *folds*) en el que, dentro de cada iteración y a partir de las matrices de entradas y salidas deseadas, mediante el vector de índices resultante de la función previa, se creen **4 matrices**: entradas y salidas deseadas para **entrenamiento** y para **prueba**. 

3. **Dentro de este bucle**, añadir una llamada para **generar los modelos**, que pueden ser cualquiera de los utilizados en la **Unidad 6**. 

4. **Entrenar esos modelos** utilizando el conjunto de entrenamiento correspondiente, es decir, los **K subconjuntos restantes** no usados para prueba.

5. En caso de necesitar un **conjunto de validación** (p. ej., para *early stopping* u otros fines), **dividir el conjunto de entrenamiento en dos partes**. Para ello, utiliza la función `holdOut`. 

4. **Construir el *ensemble*** siguiendo una de las estrategias descritas anteriormente (cualquiera de ellas) y **calcular el rendimiento en test**.  

6. Finalmente, **proporcionar el resultado del promedio** de los valores de estos vectores para cada métrica **junto con sus desviaciones estándar**. 

Como resultado de esta llamada, al menos debería devolverse **el valor en test en la(s) métrica(s) seleccionada(s)**.

In [ ]:
function trainClassEnsemble(estimators::AbstractArray{Symbol,1}, 
        modelsHyperParameters:: AbstractArray{Dict, 1},     
        trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}},    
        kFoldIndices::     Array{Int64,1})
    #TODO
end

### Pregunta
> ❓ Repite la función anterior, pero esta vez permitiendo **un único estimador** como base. Este puede **replicarse** y pasarse a la función previa.


In [ ]:
function trainClassEnsemble(baseEstimator::Symbol, 
        modelsHyperParameters::Dict,
        NumEstimators::Int=100,
        trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}},     
        kFoldIndices::     Array{Int64,1})
    #TODO
end

## Técnicas que integran el enfoque *Ensemble*

Algunos de los algoritmos más conocidos y actualmente utilizados se basan en este tipo de enfoque. Entre ellos, quizás los más representativos y ampliamente empleados son aquellos basados en la **generación de Árboles de Decisión simples (DT)**. La razón de su uso radica en su **fácil interpretación**, así como en su **rapidez de cálculo y entrenamiento**. A continuación, se presentan los dos enfoques más conocidos en este sentido: ***Random Forest*** y ***XGBoost***.

### Random Forest

El algoritmo **Random Forest**, propuesto por **Breiman y Cutler** en 2006 (a partir de una idea previa de **Ho** en 1995, conocida como *Random Subspaces*), constituye uno de los ejemplos más representativos del aprendizaje mediante *ensembles*. Combina múltiples clasificadores simples —en este caso, **Árboles de Decisión (DTs)**— en un modelo único y más robusto. Cada árbol del bosque se entrena sobre una **muestra *bootstrap*** (un subconjunto aleatorio con reemplazo) del conjunto de datos original, siguiendo un enfoque de *bagging*. Dado que cada árbol se entrena de manera independiente, el proceso puede ser completamente **paralelizado**.  

En problemas de **clasificación**, la predicción final se obtiene mediante una **votación mayoritaria** entre todos los árboles; en **regresión**, mediante el **promedio** de sus salidas.

Los **Random Forests** destacan por ofrecer un rendimiento notable con **mínimo ajuste de hiperparámetros**. Normalmente, el parámetro más importante es el **número de árboles** (`n_trees` en MLJ o `n_estimators` en *scikit-learn*), que controla el tamaño del *ensemble*. Una heurística común sugiere utilizar:

- *$\sqrt{\textrm{\#features}}$* para tareas de clasificación  
- *$\frac{\textrm{\#features}}{3}$* para tareas de regresión  

Aunque aumentar el número de árboles suele mejorar el rendimiento, este incremento tiende a **saturarse** más allá de los **500–1000 árboles** en la mayoría de los casos prácticos.

Además del proceso de *bootstrapping*, los Random Forest introducen un **segundo nivel de aleatoriedad**: en cada división de nodo, solo se considera un **subconjunto aleatorio de características** como candidatas para la partición. Esto incrementa la **diversidad entre los árboles** y contribuye a **reducir la varianza del modelo**, manteniendo al mismo tiempo una alta capacidad predictiva.  

Un subproducto relevante de este mecanismo es la posibilidad de **cuantificar la importancia de las características**. Analizando cuánto contribuye cada variable a la reducción de la impureza de los nodos a lo largo de todos los árboles, los Random Forest pueden estimar la **importancia relativa de cada característica**. Esta importancia basada en la impureza se emplea frecuentemente para **selección de características**.  
La métrica de impureza más común es el **índice de Gini**, definido como:

$$
G = \sum_{i=1}^C p(i) \cdot (1 - p(i))
$$

donde $C$ es el número de clases y $p(i)$ es la probabilidad de seleccionar aleatoriamente una instancia de la clase $i$. Intuitivamente, mide la probabilidad de clasificar incorrectamente una instancia elegida al azar si las etiquetas se asignaran de acuerdo con la distribución de clases. Para una explicación visual excelente, véase [esta referencia](https://victorzhou.com/blog/gini-impurity/).




In [ ]:
using MLJ
using MLJBase: accuracy
using Plots

# Cargar el modelo Random Forest nativo de Julia
RandomForestClassifier = @load RandomForestClassifier pkg=DecisionTree

# Definir el modelo
models["RF"] = RandomForestClassifier(
    n_trees=8,              
    max_depth=-1,           
    min_samples_split=2,
    n_subfeatures=-1,       
    sampling_fraction=1.0   
)

# Entrenar el modelo
machines["RF"] = machine(models["RF"], train_input, train_output) |> fit!

    
# Evaluar los modelos e imprimir el accuracy
for (name, mach) in machines
    acc = accuracy(predict_mode(mach, test_input), test_output)
    println("$name: $(round(acc*100, digits=2)) %")
end


### Hiperparámetros clave

| **Parámetro**         | **Descripción** |
|------------------------|-----------------|
| `n_trees`              | Número de árboles en el bosque (equivalente a `n_estimators` en *scikit-learn*). |
| `max_depth`            | Profundidad máxima de cada árbol. Usar `-1` para no establecer límite. |
| `min_samples_split`    | Número mínimo de muestras requeridas para dividir un nodo. |
| `n_subfeatures`        | Número de características aleatorias consideradas en cada división (`-1` utiliza √(#features)). |
| `sampling_fraction`    | Fracción de muestras de entrenamiento empleadas para construir cada árbol (*bootstrapping*). |
| `rng`                  | Generador de números aleatorios para garantizar la reproducibilidad. |

En este ejemplo, el número de árboles (`n_trees`) se define siguiendo la heurística de $\sqrt{\textrm{\#features}}$. Dado que el conjunto de datos utilizado en este ejemplo es relativamente pequeño, los resultados pueden **variar ligeramente entre ejecuciones**, dependiendo de las particiones aleatorias utilizadas durante el entrenamiento.

#### Importancia de características

Una vez entrenado el modelo, puede calcularse la **importancia de las características** a partir de la **reducción media de la impureza de Gini** en todos los árboles del bosque.


In [ ]:
# Extraer los parámetros de los modelos ajustados
fitted_model = fitted_params(machines["RF"])

# Obtner la importancia de las características
feature_importances = fitted_model.features_importance

# Imprimir la importancia de las características
p = bar(
    1:length(feature_importances),
    feature_importances,
    orientation = :horizontal,
    legend = false
)
xlabel!(p, "Gini Gain")
ylabel!(p, "Feature")
title!(p, "Feature Importance")

Como se muestra en la gráfica, gran parte de la información predictiva puede concentrarse en un **número reducido de características**.  
Por tanto, esta métrica también puede emplearse para la **filtración o selección de características**, aspecto que se abordará en secciones posteriores.

Los **Random Forests** representan uno de los métodos *ensemble* más **robustos y ampliamente utilizados**.  
Aprovechan el *bagging* y la **aleatoriedad en la selección de características** para construir árboles diversos, reduciendo la varianza y mejorando la capacidad de generalización.  
Además, proporcionan de forma natural medidas interpretables, como la **importancia de las características**, lo que los convierte no solo en **predictores potentes**, sino también en **herramientas útiles para el análisis exploratorio de datos**.

---

### XGBoost (*eXtreme Gradient Boosting*)

Por último, en esta sección final, debe mencionarse nuevamente el **Gradient Boosting**, en concreto una implementación que en los últimos años se ha hecho muy conocida por su **versatilidad y velocidad**. Esta implementación es conocida como ***XGBoost (eXtreme Gradient Boosting)***, y ha destacado especialmente en competiciones de plataformas como **Kaggle**, debido a su **rapidez de ejecución** y **robustez en los resultados**.  

El ***XGBoost*** constituye un *ensemble* similar al **Random Forest**, pero utiliza un clasificador base diferente denominado **CART** (*Classification and Regression Trees*), en lugar de los simples *Decision Trees*.  
Este cambio responde a la necesidad del algoritmo de obtener **probabilidades asociadas a las decisiones**, como sucede en el *Gradient Tree Boosting*. La otra diferencia fundamental de este algoritmo, al basarse en *Gradient Tree Boosting*, es el **cambio de la estrategia de *bagging* a *boosting*** para la creación de los conjuntos de entrenamiento de los clasificadores.

Posteriormente, esta técnica lleva a cabo un **entrenamiento aditivo**, cuyos pesos se ajustan mediante un **descenso por gradiente** sobre una función de pérdida (*loss function*) previamente definida. Al añadir un término de **regularización** a dicha función de pérdida, puede calcularse la **segunda derivada** de las funciones, lo que permite actualizar los pesos de clasificación de los distintos árboles. El cálculo de este gradiente posibilita ajustar los valores de los clasificadores generados posteriormente, de modo que los pesos concentren la atención en los **patrones clasificados de forma incorrecta**. Los detalles matemáticos de la implementación pueden consultarse en este [enlace](https://xgboost.readthedocs.io/en/stable/tutorials/model.html).

A diferencia de los otros enfoques vistos, **`xgboost` no está actualmente implementado dentro de `scikit-learn`** pero si en **MLJ**. Aun así en este caso **instalar la versión de referencia** si aún no se encuentra disponible en el sistema.

In [ ]:
using Pkg;
Pkg.add("XGBoost")

 Tras la instalación, la biblioteca puede utilizarse como se muestra en el siguiente ejemplo. A diferencia de otras implementaciones, la versión en **Julia** admite como entrada los formatos **Julia Array**, **SparseMatrixCSC**, **texto en formato libSVM** y **archivo binario de XGBoost**.  

Aunque las amplias opciones ofrecidas por la biblioteca de Julia permiten realizar conversiones internas al formato [LIBSVM](https://xgboost.readthedocs.io/en/stable/tutorials/input_format.html), como ocurre con otras bibliotecas, esta implementación **no soporta todas las posibilidades**. En particular, el tipo **`BitVector`** **no es actualmente compatible** con la función `DMatrix`. Por tanto, es necesario realizar un **pequeño cambio en el formato de los datos** antes de utilizar la biblioteca.


In [ ]:
using XGBoost;

train_input = input_data
train_output = output_data

test_input = input_data
test_output = output_data

train_output_asNumber= Vector{Number}(train_output);

@assert train_output_asNumber isa Vector{Number}

Una vez realizada esta adaptación de los datos, se puede proceder al **entrenamiento de un modelo** utilizando la biblioteca `xgboost`.  
Para ello, basta con llamar a la función `train` con los parámetros correspondientes.  
Entre estos parámetros, los más relevantes son:

- **eta**: término que determina la **tasa de aprendizaje** o compresión de los pesos después de cada nueva etapa de *boosting*.  
  Toma valores entre 0 y 1.  
- **max_depth**: profundidad máxima de los árboles.  
  Su valor por defecto es 6; incrementarlo permite construir modelos más complejos.  
- **gamma**: parámetro que controla la **reducción mínima de la función de pérdida** necesaria para realizar una nueva partición en una hoja del árbol.  
  Cuanto mayor sea su valor, más **conservador** será el modelo.  
- **alpha** y **lambda**: parámetros que controlan la **regularización L1 y L2**, respectivamente.  
- **objective**: define la **función de pérdida** a utilizar, que puede ser una de las predefinidas en la biblioteca.  
  La lista completa de opciones puede consultarse en este [enlace](https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster).

Además, solo es necesario establecer el **número máximo de iteraciones** del proceso de *boosting*, como se muestra en el siguiente ejemplo, con **20 rondas** de entrenamiento.

In [ ]:
svm_data = DMatrix(train_input, label=train_output_asNumber)

model = xgboost(svm_data, rounds=20, eta = 1, max_depth = 6)

En el siguiente fragmento de código, varios parámetros se pasan en forma de **diccionario**, y se calculan **dos métricas diferentes**.  La primera, *error*, se refiere al número de instancias clasificadas incorrectamente sobre el total; la segunda es el **Área Bajo la Curva ROC (AUC)**.

### Pregunta
> ❓ ¿Cuál es el nombre canónico de la primera medida que se está monitorizando?

In [ ]:
param = ["max_depth" => 2,
         "eta" => 1,
         "objective" => "binary:logistic"]
metrics = metrics = ["error", "auc"]
model = xgboost(DMatrix(train_input, label=train_output_asNumber), rounds=20, param=param, metrics=metrics)

pred = predict(model, train_input)

***Importante***

En caso de utilizar un **conjunto de validación**, este debe pasarse mediante el parámetro **`evals`** de la función de entrenamiento.  
Además, **solo cuando dicho parámetro `evals` está definido**, es posible establecer las **rondas de parada temprana** mediante el parámetro **`early_stopping_rounds`** de la función `train`.  

El código sería similar al siguiente:

```julia
evals = DMatrix(val_input, label=val_output)
xgb_model = xgb.train(param, train_input, num_round, label=train_output_asNumber, 
                      evals=evals, early_stopping_rounds=10)
```
El valor proporcionado en la salida corresponde a la **suma de las salidas de los árboles**, situándose **entre 0 y 1** para representar la **probabilidad de pertenencia a una clase dada**. Dado que se trata de un problema de clasificación binaria, basta con establecer un umbral de 0.5 sobre la salida para determinar la clase predicha.

In [ ]:
using XGBoost: predict as predict_xgb

pred = predict_xgb(model, test_input)
print("Error of XGboost= ", sum((pred .> 0.5) .!= test_output) / float(size(pred)[1]), "\n")

Finalmente, al igual que en el caso de **Random Forest**, es posible **identificar la importancia de las características** y representarla gráficamente según su posición en el **ranking de relevancia**. Con el siguiente fragmento de código puede visualizarse dicho indicador, **ordenado de forma ascendente**:


In [ ]:
feature_gain =  [(first(x),last(x)) for x in importance(model)]
feature, gain = first.(feature_gain), last.(feature_gain)

using Plots;

p = bar(feature, y=gain, orientation="h", legend=false)
xlabel!(p,"Gain")
ylabel!(p,"Feature")
title!("Feature Importance")

Como puede observarse, **no todas las características tienen la misma importancia**.  
Debe tenerse en cuenta que el eje *Feature* identifica la **posición en el vector de características**, el cual se ordena **por defecto según el valor de ganancia (*gain*)**.

## Técnicas más modernas

En los últimos años han surgido nuevas implementaciones de algoritmos basados en *ensemble learning* que buscan **optimizar el rendimiento y la eficiencia computacional** de los métodos clásicos de *Gradient Boosting*.  
Entre estas destacan **LightGBM** y **CatBoost**, dos librerías de alto rendimiento ampliamente utilizadas tanto en la investigación como en la práctica profesional.

Ambos algoritmos parten de la misma base teórica del *Gradient Boosting*, pero introducen **mejoras sustanciales en velocidad, manejo de datos categóricos y escalabilidad**.  
Estas optimizaciones los convierten en herramientas especialmente adecuadas para conjuntos de datos **grandes, complejos o heterogéneos**.  

En las siguientes secciones se presentan brevemente las características principales de **LightGBM** y **CatBoost**, junto con ejemplos prácticos de su uso e integración dentro del ecosistema **MLJ** de Julia.

#### LightGBM

**LightGBM (Light Gradient Boosting Machine)** es una implementación moderna y altamente optimizada del algoritmo de *Gradient Boosting*, desarrollada por **Microsoft Research**.  
Está diseñada específicamente para ofrecer **gran velocidad y eficiencia** en contextos con **grandes volúmenes de datos** y **alto número de características**, resolviendo algunos de los problemas de escalabilidad presentes en **XGBoost**.

A diferencia de otros algoritmos de *boosting*, LightGBM introduce dos innovaciones fundamentales:

1. **Crecimiento de los árboles orientado a hojas (Leaf-wise growth)**:  
   En lugar de crecer los árboles nivel por nivel (*level-wise*), como hace XGBoost, LightGBM expande las ramas que más reducen la pérdida (*loss*).  
   Esto produce modelos más precisos, aunque con mayor riesgo de sobreajuste si no se controla la profundidad.

2. **Histogram-based Decision Tree Learning**:  
   Los valores continuos se agrupan en histogramas, reduciendo el uso de memoria y acelerando el cálculo de los puntos de división óptimos.

#### Instalación
Para su uso en Julia, la librería se ejecuta sobre su backend en Python, por lo que puede ser más lenta en entornos no nativos.  
Se instala con:

In [ ]:
using Pkg;
Pkg.add("LightGBM")
A partir de ese punto se procederá al uso y configuración de los parámetros de la librería 
using LightGBM;

train_input = input_data
train_output = output_data

test_input = input_data
test_output = output_data

model = LGBMClassification(
    objective = "binary",
    num_iterations = 100,
    learning_rate = .1,
    early_stopping_round = 5,
    feature_fraction = .8,
    bagging_fraction = .9,
    bagging_freq = 1,
    num_leaves = 1000,
    num_class = 1,
    metric = ["auc", "binary_logloss"]
)



Como se pude ver, los parámetros más importantes son:

* **objective: "binary"**. Define la función objetivo que el modelo intentará optimizar. En este caso, "binary" indica que es un problema de clasificación binaria.
* **num_iterations: 100.**  El número total de iteraciones (o árboles) que se construirán durante el entrenamiento. Un número mayor puede mejorar la precisión, pero también puede llevar a un sobreajuste.
* **learning_rate: .1**. La tasa de aprendizaje, también conocida como "eta". Controla cuánto ajusta el modelo los pesos en cada iteración. Un valor menor puede hacer que el modelo aprenda más lentamente pero de manera más precisa.
* **early_stopping_round: 5**. Si se usa un conjunto de validación, el entrenamiento se detendrá si no hay mejora en la métrica de evaluación durante early_stopping_round iteraciones consecutivas. Esto ayuda a evitar el sobreajuste.
* **feature_fraction: .8**. El porcentaje de características (features) que se usan en cada iteración para construir un árbol. Reducir este valor puede ayudar a evitar el sobreajuste y acelerar el entrenamiento.
* **bagging_fraction: .9**. El porcentaje de datos que se utilizan en cada iteración para construir los árboles. Es una forma de realizar "bagging" y también ayuda a evitar el sobreajuste.
* **bagging_freq: 1**. La frecuencia con la que se realiza el bagging. Si se establece en 1, el bagging se realiza en cada iteración. Si se establece en un valor mayor, el bagging se realiza cada bagging_freq iteraciones.
* **num_leaves: 1000**. El número máximo de hojas en un árbol. Aumentar este valor puede permitir que el modelo capture más complejidad, pero también puede llevar a un sobreajuste.
* **num_class: 1**. El número de clases en el problema de clasificación. Para clasificación binaria, se establece en 1. Para clasificación multiclase, se establece en el número de clases.
* **metric: ["auc", "binary_logloss"]**. La lista de métricas que se utilizarán para evaluar el rendimiento del modelo. "auc" es el Área Bajo la Curva ROC, y "binary_logloss" es la pérdida logarítmica binaria, que mide la calidad de las predicciones.

Lo único que queda es entrenar y usar el modelo


In [ ]:

# Fit the estimator on the training data and return its scores for the test data.
fit!(model, train_input, train_output, (test_input, test_output))

# Predict arbitrary data with the estimator.
predict(model, test_input)

#### Integración con MLJ

LightGBM está disponible en el ecosistema de MLJ a través del paquete MLJLightGBMInterface.jl
.
Esto permite entrenar, evaluar y ajustar modelos de LightGBM siguiendo la misma estructura de machine() y fit!() utilizada en MLJ:

```julia
using MLJ, MLJLightGBMInterface

model = @load LightGBMClassifier pkg=MLJLightGBMInterface
clf = model(num_iterations=100, learning_rate=0.1, max_depth=-1)
mach = machine(clf, X, y)
fit!(mach)
```

LightGBM incluye funciones para búsqueda de hiperparámetros (`cv`, `search_cv`) y ofrece soporte tanto para CPU como GPU.
Además, los modelos pueden guardarse y cargarse fácilmente:

```julia
# Save and load the fitted model.
filename = pwd() * "/lightGBM.model"
savemodel(model, filename)
loadmodel!(model, filename)

```

### CatBoost

**CatBoost (Categorical Boosting)** es un algoritmo de *Gradient Boosting* desarrollado por **Yandex**, diseñado específicamente para tratar de forma eficiente **variables categóricas** sin necesidad de un preprocesamiento intensivo.  
Su principal fortaleza radica en que **codifica internamente las variables categóricas** mediante combinaciones estadísticas y ordenamientos aleatorios, evitando el uso de *one-hot encoding* y reduciendo el riesgo de sobreajuste.

CatBoost también introduce innovaciones que mejoran la estabilidad del modelo:

1. **Orden aleatorio controlado (*ordered boosting*)**:  
   Se evita la dependencia de la muestra durante el cálculo del gradiente, reduciendo el *prediction shift*.

2. **Combinación eficiente de características categóricas**:  
   Genera automáticamente nuevas variables basadas en combinaciones de categorías, mejorando la capacidad predictiva sin intervención manual.

#### Instalación
CatBoost se ejecuta también sobre su versión en Python y puede instalarse mediante:

In [ ]:
!pip install catboost

In [ ]:
Pkg.add("CatBoost")


Desde ese punto se puede hacer un uso similar al de LighGBM o XGboost, soportando varios formatos.

In [ ]:
using CatBoost;
using PythonCall;

train_input = input_data
train_output = output_data

test_input = input_data
test_output = output_data

model = CatBoostclassifier(iterations = 2, learning_rate = 1, depth = 2)

Una vez definido el modelo este puede entrenarse y usarse como sigue

In [ ]:

# Fit model
fit!(model, train_data, train_labels)

# Get predictions
preds = predict(model, eval_data)

Principales parámetros

* iterations: número de iteraciones o árboles (por defecto 1000).

* learning_rate: tasa de aprendizaje, típicamente entre 0.01 y 0.1.

* depth: profundidad máxima de los árboles.

* l2_leaf_reg: regularización L2.

* eval_metric: métrica de evaluación (por ejemplo, AUC, Accuracy).

* early_stopping_rounds: número de iteraciones sin mejora antes de detener el entrenamiento.

* bootstrap_type: método de bootstrap (Bayesian, Bernoulli, Poisson, etc.).

* task_type: define si se entrena en CPU o GPU.

Integración con MLJ

CatBoost también se encuentra disponible en MLJ mediante el paquete MLJCatBoostInterface.jl
.
Esto permite utilizarlo en flujos MLJ de forma homogénea:
``` julia
using MLJ, MLJCatBoostInterface

model = @load CatBoostClassifier pkg=MLJCatBoostInterface
clf = model(iterations=500, learning_rate=0.05, depth=8)
mach = machine(clf, X, y)
fit!(mach)
```





## Notas sobre Julia

### Comprendiendo `coerce` en MLJ

Al trabajar con conjuntos de datos en **MLJ**, es importante entender que el sistema distingue entre los **tipos de máquina** (por ejemplo, `Int64`, `Float64`, `String`) y los **tipos científicos** (*scientific types* o *scitypes*), que describen cómo deben **interpretarse los datos** en el contexto del modelado.

### Por qué se necesita `coerce`

Los modelos de **MLJ** no se basan en los tipos nativos de Julia, sino que esperan que las variables tengan **significados científicos** explícitos:
- Una columna numérica puede representar una característica **continua**.  
- Una columna de texto (*string*) puede representar una característica **categórica**.  
- Una columna booleana o entera puede ser **ordenada** o **no ordenada**, según el contexto.

Dado que Julia no puede inferir esto automáticamente, utilizamos la función `coerce()` para **indicar explícitamente a MLJ cómo debe interpretar cada columna**.  
De este modo, se garantiza la **compatibilidad entre los datos y el modelo de MLJ** que se desea emplear.

---

### Ejemplo



In [ ]:
using MLJ, DataFrames, CSV

# Carrgar el conjunto de datos Sonar
data = CSV.read("sonar.csv", DataFrame)

# Inspeccionar los tipos de columna actuales
schema(data)

# Supongamos que la última columna es el objetivo ('Rock' o 'Mine')
y, X = unpack(data, ==(:Target), rng=123)

# Convertir la variable objetivo a categórica
y = coerce(y, Multiclass)

# Convertir todas las columnas de características a continuas
X = coerce(X, autotype(X, rules = (:discrete_to_continuous,)))

# Verificar los nuevos tipos
schema(X)

### Ejemplos comunes

| Situación                             | Qué hacer                | Ejemplo                               |
| ------------------------------------- | ------------------------ | ------------------------------------- |
| Etiquetas categóricas almacenadas como cadenas | Convertir a `Multiclass` | `y = coerce(y, Multiclass)`           |
| Características numéricas             | Convertir a `Continuous` | `X = coerce(X, :var1 => Continuous)`  |
| Inferencia automática                 | Usar `autotype()`        | `autotype(X)`                         |
| Comprobar los tipos científicos actuales | Usar `schema()`          | `schema(X)`                           |

### Qué ocurre si se omite `coerce`

Si se omite el paso de coerción:

* **MLJ puede interpretar incorrectamente la variable objetivo** como continua, impidiendo el uso de modelos de clasificación.  
* **Algunos algoritmos** (por ejemplo, *DecisionTree*, *NaiveBayes*) pueden **fallar o generar predicciones incorrectas**.  
* La función `machine()` puede **rechazar la vinculación entre el modelo y los datos** si los tipos no son compatibles.


## Voting Classifier

Siguiendo estas líneas, se presentan varios ejemplos de cómo utilizar el nuevo **Voting Classifier** que hemos implementado.

En ejemplos previos, construimos el *ensemble* de la siguiente manera:

```julia
voting_hard = VotingClassifier(models=base_models_list, voting=:hard)
voting_soft = VotingClassifier(models=base_models_list, voting=:soft)

mach_hard = machine(voting_hard, train_input, train_output) |> fit!
mach_soft = machine(voting_soft, train_input, train_output) |> fit!
```

* `voting=:hard` → utiliza votación mayoritaria (basada en las etiquetas de clase).  

* `voting=:soft` → utiliza votación por promedio de probabilidades (requiere modelos que generen probabilidades como salida).  

Los siguientes ejemplos muestran cómo **modificar e integrar dinámicamente** el `VotingClassifier` dentro de tu flujo de trabajo.


In [ ]:
### Ejemplo 1: Cambio dinámico del tipo de votación
ensemble = VotingClassifier(models=base_models_list, voting=:hard)
println("\nTipo de votación actual: $(ensemble.voting)")

ensemble.voting = :soft
println("Tipo de votación actual: $(ensemble.voting)
")


In [ ]:
# Ejemplo 2: Uso del `VotingClassifier` en una *Pipeline*
pipe_hard = @pipeline(
    Standardizer(),
    VotingClassifier(models=base_models_list, voting=:hard)
)

pipe_soft = @pipeline(
    Standardizer(),
    VotingClassifier(models=base_models_list, voting=:soft)
)

In [ ]:
# Ejemplo 3: Validación cruzada comparando ambas estrategias de votación
cv_hard = evaluate!(
    machine(voting_hard, train_input, train_output),
    resampling=CV(nfolds=5),
    measure=accuracy
)
println("Hard Voting CV accuracy: $(round(mean(cv_hard.measurement)*100, digits=2)) %")

cv_soft = evaluate!(
    machine(voting_soft, train_input, train_output),
    resampling=CV(nfolds=5),
    measure=accuracy
)
println("Soft Voting CV accuracy: $(round(mean(cv_soft.measurement)*100, digits=2)) %")